<a href="https://colab.research.google.com/github/aknip/Local_LLMs/blob/main/Mixtral_8x7B_llama.cpp_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Mixtral-8x7B-Instruct on Free Colab: Slow 4t/s But Works

- It takes about 10 minutes for everything to be setup, downloaded and loaded.

- Wait until the inference cell outputs "llama server listening at... all slots are idle and system prompt is empty, clear the KV cache." Use the URL from the output of the previous cell that says "Connect using the following address when the server is up."


Uses llama.cpp https://github.com/ggerganov/llama.cpp

Source: https://www.reddit.com/r/LocalLLaMA/comments/18ggkp6/mixtral8x7binstruct_on_free_colab_slow_4ts_but/

In [1]:
#@title #Choose a Model
model = 'mixtral-8x7b-instruct-v0.1.Q2_K' #@param ['mixtral-8x7b-instruct-v0.1.Q2_K', 'dolphin-2.6-mixtral-8x7b.Q2_K', 'mistral-7b-instruct-v0.2.Q8', 'dolphin-2.2.1-mistral-7b.Q8']


In [2]:
#@title #Runtime Info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


Thu Jan  4 11:59:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
#@title # Setup
%cd /content/
!rm -rf llama.cpp
!git clone --depth 1 https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp
!make LLAMA_CUBLAS=1

if model == 'mixtral-8x7b-instruct-v0.1.Q2_K':
	!wget https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q2_K.gguf
elif model == 'dolphin-2.6-mixtral-8x7b.Q2_K':
	!wget https://huggingface.co/TheBloke/dolphin-2.6-mixtral-8x7b-GGUF/resolve/main/dolphin-2.6-mixtral-8x7b.Q2_K.gguf
elif model == 'mistral-7b-instruct-v0.2.Q8':
	!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q8_0.gguf
elif model == 'dolphin-2.2.1-mistral-7b.Q8':
	!wget https://huggingface.co/TheBloke/dolphin-2.2.1-mistral-7B-GGUF/resolve/main/dolphin-2.2.1-mistral-7b.Q8_0.gguf


/content
Cloning into 'llama.cpp'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (420/420), done.
remote: Compressing objects: 100% (384/384), done.
remote: Total 420 (delta 44), reused 240 (delta 12), pack-reused 0
Receiving objects: 100% (420/420), 6.53 MiB | 5.29 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/content/llama.cpp
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include -I/usr/local/cuda/targets/aarch64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_U

In [4]:
#@title # Connect using the following address when the server is up.
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(12345)"))

https://kyjnizozvil-496ff2e9c6d22116-12345-colab.googleusercontent.com/


In [5]:
#@title # inference
%cd /content/llama.cpp
if model == 'mixtral-8x7b-instruct-v0.1.Q2_K':
	!./server -m mixtral-8x7b-instruct-v0.1.Q2_K.gguf -ngl 27 -c 2048 --port 12345
elif model == 'dolphin-2.6-mixtral-8x7b.Q2_K':
	!./server -m dolphin-2.6-mixtral-8x7b.Q2_K.gguf -ngl 27 -c 2048 --port 12345
elif model == 'mistral-7b-instruct-v0.2.Q8':
	!./server -m mistral-7b-instruct-v0.2.Q8_0.gguf -ngl 35 -c 0 --port 12345
elif model == 'dolphin-2.2.1-mistral-7b.Q8':
	!./server -m dolphin-2.2.1-mistral-7b.Q8_0.gguf -ngl 35 -c 0 --port 12345



/content/llama.cpp
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
{"timestamp":1704370078,"level":"INFO","function":"main","line":2776,"message":"build info","build":1,"commit":"a919280"}
{"timestamp":1704370078,"level":"INFO","function":"main","line":2779,"message":"system info","n_threads":2,"n_threads_batch":-1,"total_threads":2,"system_info":"AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | "}
llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from mixtral-8x7b-instruct-v0.1.Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.ar